## Instalar imutils

In [ ]:
!pip install imutils

## Importar paquetes necesarios

- `imutils` provee una serie de funciones (utils) para el procesamiento de imágenes.
- `structural_similarity`: nos servirá para realizar la comparación usando [structural similarity](https://es.abcdef.wiki/wiki/Structural_similarity).
- `cv2` es OpenCV para Python. Nos ayudará a abrir la imagen y realizar las operaciones necesarias para la detección de colores.

In [ ]:
from skimage.metrics import structural_similarity as compare_ssim
import imutils
import cv2

## Cargar imágenes y convertir escala de grises

La conversión a escala de grises se da para simplificar la imagen a la hora de hacer la comparación.

In [ ]:
# Cargar imágenes
imagen_A = cv2.imread("bad_card.jpg")
imagen_B = cv2.imread("good_card.jpg")

# Convertir a escala de grises
gris_A = cv2.cvtColor(imagen_A, cv2.COLOR_BGR2GRAY)
gris_B = cv2.cvtColor(imagen_B, cv2.COLOR_BGR2GRAY)

## Comparación
La comparación se hace con `compare_ssim`. Note que esa función retorna un `puntaje` y la `diferencia`. `puntaje` realmente representa el SSIM (medida del índice de similitud estructural). En otras palabras, es la diferencia entre dos imágenes. Ese valor puede estar entre [-1, 1], donde un valor de 1 sería que las imágenes calzan perfectamente y, por ende, no hay diferencia alguna. `diferencia` es un array de flotantes entre [-1, 1] que representa las diferencias entre las imágenes.

In [ ]:
(puntaje, array_diferencia) = compare_ssim(gris_A, gris_B, full=True)

# Convertir de array de flotantes a enteros sin signo de 8 bits para luego procesarlo con OpenCV.
imagen_diferencia = (array_diferencia * 255).astype("uint8")

print("SSIM: {}".format(puntaje))

## Obtener las regiones donde hay diferencias

`imagen_thresh` representa una imagen que se le aplicó una técnica de [thresholding ](https://es.wikipedia.org/wiki/M%C3%A9todo_del_valor_umbral). Con los parámetros aplicados, se desplegarán en blanco únicamente las diferencias entre las dos imágenes. Todo lo que sea igual será negro. `regiones` son los sectores donde se ubican las diferencias entre las dos imágenes.

In [ ]:
imagen_thresh = cv2.threshold(imagen_diferencia, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

regiones = cv2.findContours(imagen_thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
regiones = imutils.grab_contours(regiones)

## Desplegar imágenes

Se pintan rectángulos rojos en cada ventana donde se encuentren las diferencias entre las dos imágenes. Luego, se despliegan las 4 imágenes creadas.

In [ ]:
# Iterar sobre las regiones para pintar las diferencias
for region in regiones:
    (x, y, w, h) = cv2.boundingRect(region)
    cv2.rectangle(imagen_A, (x, y), (x + w, y + h), (0, 0, 255), 2)
    cv2.rectangle(imagen_B, (x, y), (x + w, y + h), (0, 0, 255), 2)
    
# Desplegar imagenes
cv2.imshow("Original", imagen_A)
cv2.imshow("Falsa", imagen_B)
cv2.imshow("Diferencia", imagen_diferencia)
cv2.imshow("Thresh", imagen_thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()